In [ ]:
!pip install transformers datasets evaluate

In [ ]:
# если gpu, то это не устанавливать !pip install optimum[onnxruntime]

In [ ]:
#  важно! только с gpu!!!
!pip install optimum[onnxruntime-gpu]

In [3]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import pandas as pd

from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from evaluate import evaluator
import evaluate

#**Сonvert to ONNX**

In [5]:
from optimum.onnxruntime import ORTModelForSequenceClassification

model_checkpoint = "lvwerra/distilbert-imdb"
save_directory = "onnx/"

# Load a model from transformers and export it to ONNX
ort_model = ORTModelForSequenceClassification.from_pretrained(model_checkpoint, export=True)
#tokenizer = AutoTokenizer.from_pretrained("lvwerra/distilbert-imdb")
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Save the onnx model and tokenizer
ort_model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

Framework not specified. Using pt to export to ONNX.


Using the export variant default. Available variants are:
	- default: The default ONNX variant.
Using framework PyTorch: 2.1.0+cu118
/usr/local/lib/python3.10/dist-packages/transformers/models/distilbert/modeling_distilbert.py:223: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)


('onnx/tokenizer_config.json',
 'onnx/special_tokens_map.json',
 'onnx/vocab.txt',
 'onnx/added_tokens.json',
 'onnx/tokenizer.json')

In [6]:
clf_metrics = evaluate.combine(["accuracy",
                                "f1",
                                "precision",
                                "recall"])
cuda_pipe = pipeline("text-classification",
                model=ort_model,
                tokenizer=tokenizer,
                device='cuda')

data = load_dataset("imdb",
                    split="test").shuffle().select(range(1000))
task_evaluator = evaluator("text-classification")

cuda_results = task_evaluator.compute(model_or_pipeline=cuda_pipe,
                                 data=data,
                                 metric=clf_metrics,
                                 label_mapping={"NEGATIVE": 0, "POSITIVE": 1},)
print('CUDA RESULTS')
print(cuda_results)

use_io_binding was set to False, setting it to True because it can provide a huge speedup on GPUs. It is possible to disable this feature manually by setting the use_io_binding attribute back to False.


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

CUDA RESULTS
{'accuracy': 0.933, 'f1': 0.9348882410106901, 'precision': 0.9321705426356589, 'recall': 0.9376218323586745, 'total_time_in_seconds': 9.945266764999985, 'samples_per_second': 100.55034456383449, 'latency_in_seconds': 0.009945266764999985}


результат в таблице https://docs.google.com/spreadsheets/d/1raJc55pfesADS5l5FFFuZklGGY3s23ml3yISpkk02g0/edit?usp=sharing